# Problem 10.3: Controlling p53 levels


<hr>

In [1]:
import collections
import numpy as np

<hr>

[p53](https://en.wikipedia.org/wiki/P53) is an important protein for tumor suppression, DNA damage repair, cell cycle regulation, and more. In their [2012 paper](https://doi.org/10.1126/science.1218351), the researchers in Galit Lahav's group investigated how temporal dynamics of p53 controls cell fate in [MCF-7 cells](https://en.wikipedia.org/wiki/MCF-7), a breast cancer cell line. It had been previously observed that p53 levels oscillate upon exposure to stress due to γ radiation. To instead study how cells respond to sustained levels of p53, the authors controlled p53 levels using the drug Nutlin-3. To do this, they devised a program for adjusting Nutlin-3 levels in the cell culture that would keep the p53 levels at a constant high level. They based this program on a mathematical model of the p53 circuit (shown in the figure below) that they developed and parametrized in an [earlier paper](https://doi.org/10.1016/j.molcel.2008.03.016).

<div style="margin: auto; width: 300px">
    
![p53 circuit](p53_circuit.svg)

</div>

The p53 levels oscillate due to a delays in the autoinhibitory loops mediated by Mdm2 and Wip1. The dynamical equations for this circuit, as presented in the paper, are

\begin{align}
\frac{\mathrm{d}p_i}{\mathrm{d}t} &= \beta_\mathrm{p} - \frac{\delta_\mathrm{mp}^i \, m\,p_i}{1+\left(n(t-\tau_\mathrm{n})/k\right)^{n_\mathrm{n}}} - \beta_\mathrm{sp}\,p_i\,\frac{(s/k_\mathrm{s})^{n_\mathrm{s}}}{1 + (s/k_\mathrm{s})^{n_\mathrm{s}}} - \gamma_\mathrm{p}^i\, p_i,\\[1em]
\frac{\mathrm{d}p_a}{\mathrm{d}t} &=  \beta_\mathrm{sp}\,p_i\,\frac{(s/k_\mathrm{s})^{n_\mathrm{s}}}{1 + (s/k_\mathrm{s})^{n_\mathrm{s}}} - \frac{\delta_\mathrm{np}^a \, m\,p_a}{1+\left(n(t-\tau_\mathrm{n})/k\right)^{n_\mathrm{n}}},\\[1em]
\frac{\mathrm{d}m}{\mathrm{d}t} &= \beta_{m_i} + \beta_\mathrm{m}\,p_a(t-\tau_\mathrm{m}) - \delta_\mathrm{sm}\,s\,m - \gamma_m\,m,\\[1em]
\frac{\mathrm{d}w}{\mathrm{d}t} &= \beta_\mathrm{w}\,p_a(t-\tau_\mathrm{w}) - \gamma_\mathrm{w}\,w,\\[1em]
\frac{\mathrm{d}s}{\mathrm{d}t} &= \beta_\mathrm{s}\,\theta(t) - \delta_\mathrm{ws}\,s\,\frac{(w/k_\mathrm{w})^{n_\mathrm{w}}}{1 + (w/k_\mathrm{w})^{n_\mathrm{w}}} - \gamma_\mathrm{s}\,s.
\end{align}

Here, $p_i$ and $p_a$ denote the concentrations of inactive and active p53, respectively. The variables $m$ and $w$ respectively represent the concentrations of Mdm2 and Wip1. The concentration of the DNA damage signal is denoted as $s$. Finally, $n$ is the externally imposed concentration of Nutlin-3. The function $\theta(t)$ is the Heaviside function,

\begin{align}
\theta(t) = \left\{\begin{array}[ll] 00 &  t < 0 \\
1 &  t \ge 0.
\end{array}
\right.
\end{align}

The parameters for the model are given in the code cell below as a named tuple for convenience. The units of all parameters are such that time is in hours and concentrations in micromolar. Before proceeding, take a moment and make sure you understand the physical meaning of each term in the equations.

In [2]:
Params = collections.namedtuple(
    "Params",
    [
        "delta_mpi",
        "gamma_pi",
        "delta_mpa",
        "delta_sm",
        "gamma_m",
        "gamma_w",
        "delta_ws",
        "gamma_s",
        "beta_p",
        "beta_sp",
        "beta_m",
        "beta_mi",
        "beta_w",
        "beta_s",
        "tau_n",
        "tau_m",
        "tau_w",
        "k",
        "n_n",
        "n_s",
        "n_w",
        "k_w",
        "k_s",
    ],
)

params = Params(
    delta_mpi=5,
    gamma_pi=2,
    delta_mpa=1.4,
    delta_sm=0.5,
    gamma_m=1,
    gamma_w=0.7,
    delta_ws=50,
    gamma_s=7.5,
    beta_p=0.9,
    beta_sp=10,
    beta_m=0.9,
    beta_mi=0.2,
    beta_w=0.25,
    beta_s=10,
    tau_n=0.4,
    tau_m=0.7,
    tau_w=1.25,
    k=2.3,
    n_n=3.3,
    n_s=4,
    n_w=4,
    k_w=0.2,
    k_s=1,
)

**a)** Solve the equations numerically with $n = 0$. Plot the solution along with the experimental measurements from the [Purvis, et al. paper](https://doi.org/10.1126/science.1218351), given in the Numpy arrays below. This helps verify that the model gives results similar to what you would observe experimentally.

In [3]:
# Time points (in units of hours)
time = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 12])

# p53 concentration
relative_total_p53_conc = np.array(
    [0.13, 0.69, 1.00, 0.87, 0.60, 0.35, 0.34, 0.54, 0.61, 0.42]
)

**b)** Come up with a program for controlling the Nutlin-3 concentration so as to bring the total p53 level to a sustained value similar to that of its maximum value during oscillations in the absence of Nutlin-3. I.e., choose a function $N(t)$ to give a sustained level of p53 that is approximately unity (in the units specified by the parameter values). You might want to consider experimental constraints; for example that you might want to have only a few steps where the Nutlin-3 concentrations are adjusted to ease experimentation. Show a plot verifying that your scheme works.

<br />

## Solution

<hr>

In [5]:
import biocircuits

import bokeh.io
import bokeh.plotting

# import panel as pn
# pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

Because we want to solve the differential equations with steps in Nutlin concentration (steps are easier to handle experimentally, requiring a simple injection), we will include a function `n_fun()` which defines the time points and levels of steps of Nutlin.

In [6]:
def n_fun(t, t_step, levels):
    """Steps and values of Nutlin. len(levels) = len(t_step) + 1."""
    return levels[np.searchsorted(t_step, t, "right")]

Next, we need to code up a function to compute the right hand side of the DDEs for the p53 system.

In [7]:
def dy_dt(
    y,
    t,
    y_past,
    n_fun,
    t_step=(0,),
    levels=(0, 0),
    delta_mpi=5,
    gamma_pi=2,
    delta_mpa=1.4,
    delta_sm=0.5,
    gamma_m=1,
    gamma_w=0.7,
    delta_ws=50,
    gamma_s=7.5,
    beta_p=0.9,
    beta_sp=10,
    beta_m=0.9,
    beta_mi=0.2,
    beta_w=0.25,
    beta_s=10,
    tau_n=0.4,
    tau_m=0.7,
    tau_w=1.25,
    k=2.3,
    n_n=3.3,
    n_s=4,
    n_w=4,
    k_w=0.2,
    k_s=1,
):
    """
    Dynamics of x, dependent on a past and current value of x.
    """
    # Unpack
    p_i, P_a, M, W, S = y

    # Compute past values
    _, P_a_past_M, _, _, _ = y_past(t - tau_m)
    _, P_a_past_W, _, _, _ = y_past(t - tau_w)
    N_past = n_fun(t - tau_n, t_step, levels)

    # Derivatives
    dPi_dt = (
        beta_p
        - delta_mpi * biocircuits.rep_hill(N_past / k, n_n) * M * p_i
        - beta_sp * p_i * biocircuits.act_hill(S / k_s, n_s)
        - gamma_pi * p_i
    )

    dPa_dt = (
        beta_sp * p_i * biocircuits.act_hill(S / k_s, n_s)
        - delta_mpa * biocircuits.rep_hill(N_past / k, n_n) * M * P_a
    )

    dM_dt = beta_m * P_a_past_M + beta_mi - delta_sm * S * M - gamma_m * M

    dW_dt = beta_w * P_a_past_W - gamma_w * W

    dS_dt = (
        beta_s * (t >= 0)
        - delta_ws * biocircuits.act_hill(W / k_w, n_w) * S
        - gamma_s * S
    )

    return np.array([dPi_dt, dPa_dt, dM_dt, dW_dt, dS_dt])

Finally, we need to provide a function to compute the concentration of the respective species at time $t < 0$. In other words, we need to specify the initial condition. We will start with $p_a = 0$ and $p_i = 0.13$, which is the initial amount of p53. We also take the initial amount of M to be 0.2 (but it does not really matter, we could take it to be zero).

In [12]:
def y0(t, pi_0=0.2, pa_0=0.0, m_0=0.2, w_0=0, s_0=0):
    return np.array([pi_0, pa_0, m_0, w_0, s_0])

**a)** We can now use the DDE integrator in biocircuits to perform the integration. We will plot the result for $n = 0$ with the experimental data.

In [13]:
# Nutlin concentration always at zero
t_step = (0,)
levels = (0, 0)

# Set up time points and solve
t = np.linspace(0, 12, 100)
y = biocircuits.ddeint(
    dy_dt,
    y0,
    t,
    (params.tau_n, params.tau_m, params.tau_w),
    args=(n_fun, t_step, levels, *params),
)

# Make a plot
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=200,
    x_axis_label="time (hr)",
    y_axis_label="p53 level (a.u.)",
)
p.line(t, y[:, 1] + y[:, 0], line_width=2)
p.circle(time, relative_total_p53_conc, color="orange")

bokeh.io.show(p)

The agreement is ok. The paper uses giant square symbols to make it look better, but theirs is really no better than this.

**b)** We will build a dashboard to mess around with Nutlin injections to bring us to a steady level of total p53 of one. We will limit ourselves to at most three injections of Nutlin for experimental convenience.

In [9]:
level_1_slider = pn.widgets.FloatSlider(
    name="level 1 (µM)", start=0, end=10, value=0
)
level_2_slider = pn.widgets.FloatSlider(
    name="level 2 (µM)", start=0, end=10, value=0
)
level_3_slider = pn.widgets.FloatSlider(
    name="level 3 (µM)", start=0, end=10, value=0
)
time_1_slider = pn.widgets.FloatSlider(
    name="time injection time (hr)", start=0, end=12, value=0
)
time_2_slider = pn.widgets.FloatSlider(
    name="time injection time (hr)", start=1, end=12, value=0
)
time_3_slider = pn.widgets.FloatSlider(
    name="time injection time (hr)", start=2, end=12, value=0
)


@pn.depends(
    level_1_slider.param.value,
    level_2_slider.param.value,
    level_3_slider.param.value,
    time_1_slider.param.value,
    time_2_slider.param.value,
    time_3_slider.param.value,
)
def plot_p53(level_1, level_2, level_3, time_1, time_2, time_3):
    # Get levels and ordering of injections
    t_step = np.array([time_1, time_2, time_3])
    order = np.argsort(t_step)
    t_step = t_step[order]
    levels = np.concatenate(
        ((0,), np.array([level_1, level_2, level_3])[order])
    )

    # Solve
    t = np.linspace(0, 12, 100)
    y = biocircuits.ddeint(
        dy_dt,
        y0,
        t,
        (params.tau_n, params.tau_m, params.tau_w),
        args=(n_fun, t_step, levels, *params),
    )

    # Make a plot
    p = bokeh.plotting.figure(
        frame_width=400,
        frame_height=200,
        x_axis_label="time (hr)",
        y_axis_label="p53 level (a.u.)",
    )
    p.line(t, y[:, 1] + y[:, 0], line_width=2)

    return p


pn.Row(
    plot_p53,
    pn.Spacer(width=15),
    pn.Column(
        level_1_slider,
        level_2_slider,
        level_3_slider,
        time_1_slider,
        time_2_slider,
        time_3_slider,
    ),
)

Row
    [0] ParamFunction(function)
    [1] Spacer(width=15)
    [2] Column
        [0] FloatSlider(end=10, name='level 1 (µM)')
        [1] FloatSlider(end=10, name='level 2 (µM)')
        [2] FloatSlider(end=10, name='level 3 (µM)')
        [3] FloatSlider(end=12, name='time injection t...)
        [4] FloatSlider(end=12, name='time injection t..., start=1)
        [5] FloatSlider(end=12, name='time injection t..., start=2)

In playing with the sliders, we find that if we raise the level of Nutlin to 3.5 µM at 2.5 hours and then again to 5.7 µM at 4.1 hours (no third injection is necessary), we can achieve the desired effect.

## Computing environment

In [ ]:
%load_ext watermark
%watermark -v -p numpy,biocircuits,bokeh,panel,jupyterlab